# Plotting Fire Data on a Bubble Map 

## Using Plotly for Python 

[Plotly](https://plot.ly/#/) has a [Python library](https://plot.ly/python/) that allows you to create interactive data visualizations online, such as line plots, scatter maps, heatmaps, and 3D network graphs. In this tutorial, we will create a bubble map that visualizes wildfire data in California (based on the FIRMS data we acquired). 

Follow the [Getting Started](https://plot.ly/python/getting-started/) tutorial to set up the Plotly.py library. Once that is done, let's import some libraries that we will use in this tutorial:

In [1]:
import plotly.plotly as py
#import pandas as pd
from datascience import *
import numpy as np

Our FIRMS dataset is quite big, so let's narrow it down to just a few points on the map. I'd like to plot the 4 brightest, 4 most "mediocre"ly bright, and 4 dimmest wildfires in the dataset. 

First, let's load the .csv file into a table:

In [4]:
data=Table().read_table('~/URAP/google_earth_fires/data/SoCal_fires.csv') #from https://github.com/jupyterhub/jupyterhub/issues/376
data

latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,type
36.234,-118.43,321.9,1,1,2017-10-07,1845,Terra,MODIS,44,6.1,302.5,14.5,0
36.258,-118.421,319.4,4.7,2,2017-10-15,1756,Terra,MODIS,72,6.1,294.9,138.7,0
36.259,-118.425,318.2,4.7,2,2017-10-15,1756,Terra,MODIS,44,6.1,294.4,118.3,0
36.256,-118.42,345.1,1.1,1,2017-10-15,2112,Aqua,MODIS,92,6.1,298.3,50,0
36.272,-118.429,326.3,1.1,1,2017-10-16,1839,Terra,MODIS,71,6.1,293,23.1,0
36.271,-118.418,331.1,1.1,1,2017-10-16,1839,Terra,MODIS,83,6.1,292.7,29.9,0
36.255,-118.412,300,3,1.6,2017-10-16,913,Aqua,MODIS,5,6.1,281.5,31.6,0
36.256,-118.428,327,4,1.9,2017-10-16,2017,Aqua,MODIS,77,6.1,278.8,192.8,0
36.253,-118.428,330.3,1.4,1.2,2017-10-17,542,Terra,MODIS,100,6.1,281.1,49.7,0
36.412,-119.232,339.2,1.7,1.3,2017-09-08,1916,Terra,MODIS,87,6.1,302.5,73.2,0


## Filtering the data

Now, let's select our columns of interest. For the purposes of this tutorial, we care about the latitude, longitude, brightness, and dates of our data.

In [47]:
py_data=data.select(0,1,2,5) #get the columns you want
py_data

latitude,longitude,brightness,acq_date
36.234,-118.43,321.9,2017-10-07
36.258,-118.421,319.4,2017-10-15
36.259,-118.425,318.2,2017-10-15
36.256,-118.42,345.1,2017-10-15
36.272,-118.429,326.3,2017-10-16
36.271,-118.418,331.1,2017-10-16
36.255,-118.412,300,2017-10-16
36.256,-118.428,327,2017-10-16
36.253,-118.428,330.3,2017-10-17
36.412,-119.232,339.2,2017-09-08


Next, let's find the top 5 brightest fires in our data. To do this, we can sort the brightness from brightest to dimmest, and use the numpy.take method to grab the first 5 rows. You will also do the same thing to find the 5 dimmest fires, except you will sort from dimmest to brightest.

In [6]:
brightest=py_data.sort('brightness', descending=True).take(range(0,5))
brightest

latitude,longitude,brightness,acq_date
34.466,-119.078,504,2017-12-06
34.47,-119.652,503.7,2017-12-16
34.549,-119.35,502.4,2017-12-11
34.415,-119.41,502.3,2017-12-10
34.477,-119.079,501.9,2017-12-06


In [7]:
dimmest=py_data.sort('brightness',descending=False).take(range(0,5))
dimmest

latitude,longitude,brightness,acq_date
36.255,-118.412,300,2017-10-16
34.339,-118.32,300,2017-12-06
37.681,-119.592,300,2017-10-08
37.212,-119.418,300,2017-12-06
36.693,-119.7,300,2018-01-26


Finding the middle 5 fires will take a bit more work. First, we will find the median of our brightness values. Then, we will find the index of table in which the median is located. Finally, we will want to take the two rows above the median and the two rows below the median to get the middle 5 of the table. 

In [8]:
bright_col=py_data.column('brightness') #the brightness column of our data
medium=np.median(bright_col) #find median of brightnesses
list(bright_col).index(medium) #find index of median
middle=py_data.take(range((659-2),(659+3))) #take two above and two below median to get "middle 5 brightest"

In [39]:
middle

latitude,longitude,brightness,acq_date
37.562,-119.142,333,2018-06-25
37.56,-119.13,348.3,2018-06-25
37.557,-119.116,357.1,2018-06-25
37.552,-119.144,349.9,2018-06-25
37.55,-119.13,332.7,2018-06-25


Now, it's time to concatenate all our data! I am going to create arrays for each variable that we are interested in, using data from our "brightest", "middle", and "dimmest" tables.

In [9]:
lats=np.append(np.append(brightest.column(0),(middle.column(0))), dimmest.column(0))
longs=np.append(np.append(brightest.column(1),(middle.column(1))), dimmest.column(1))
brights=np.append(np.append(brightest.column(2),(middle.column(2))), dimmest.column(2))
dates=np.append(np.append(brightest.column(3),(middle.column(3))), dimmest.column(3))

Now, let's create a new table with columns for these variables! This table will contain the latitude, longitude, brightness, and dates of the top 5 brightest, middle 5, and 5 dimmest fire data. 

In [10]:
filtered_data=Table().with_columns(
        'latitude', lats,
        'longitude', longs,
        'brightness', brights,
        'acq_date', dates
        )
filtered_data

latitude,longitude,brightness,acq_date
34.466,-119.078,504,2017-12-06
34.47,-119.652,503.7,2017-12-16
34.549,-119.35,502.4,2017-12-11
34.415,-119.41,502.3,2017-12-10
34.477,-119.079,501.9,2017-12-06
37.562,-119.142,333,2018-06-25
37.56,-119.13,348.3,2018-06-25
37.557,-119.116,357.1,2018-06-25
37.552,-119.144,349.9,2018-06-25
37.55,-119.13,332.7,2018-06-25


## Plotting the data

Here's where things get a bit tricky. Ultimately, we would like to plot a figure with a specific set of data and a specific layout. Plotly has many attributes to help us accomplish our goal, but how do we know which ones to use? Luckily, Plotly has a [figure reference!](https://plot.ly/python/reference/)

Let's initialize our data first. We want to have different-colored bubbles on our bubble map so that we can differentiate between the three levels of brightness. Let's create a list of colors to choose from, using their RGB values: 

In [ ]:
colors = ["rgb(255,0,0)", "rgb(255,128,0)", "rgb(255,255,0)"] #red, orange, yellow

Next, we'll create a list called limits. Limits contains three elements, and each element contains the beginning index and ending index of a brightness level. This list will allow us to map certain attributes to a specific range of data. For example, if we assign the first element of [colors] to the first element in [limits], the 5 brightest fires will appear as red bubbles on the map.

In [1]:
limits=[(0,5),(5,10),(10,15)] #we are grouping by brightness, where first five (0,5) are brightest, second group of five (5,10) are medium, etc.

Now, it's time to create a list that contains all the information that we want to represent on the map. We'll call this list "fires." In the end, we want this list to hold three dictionaries (one for each brightness level). 

Since we have three brightness levels, we want to assign specific attributes to each of them. To do this, we will iterature through each of the groups and create a dictionary of attributes for each. If you are unclear about these attributes, refer to the [figure reference!](https://plot.ly/python/reference/).

In [2]:

#colors = ["rgb(255,0,0)", "rgb(255,128,0)", "rgb(255,255,0)"] #red, orange, yellow
#limits=[(0,5),(5,10),(10,15)] #we are grouping by brightness, where first five (0,5) are brightest, second group of five (5,10) are medium, etc.

fires=[] #the data that we want to represent on the map
for i in range(len(limits)): #we want to iterate 3 times 
    group=filtered_data.take(range(limits[i][0], limits[i][1])) #i.e: in the first iteration, we are taking the top 5 rows (the brightest) 
    fire=dict(
        type = 'scattergeo', #the type of figure we want to create 
        locationmode = 'USA-states', 
        lon = group['longitude'],
        lat = group['latitude'],
        location=['California'], #this doesn't do anything
        #text = three_sub['text'],
        marker=dict(
                size=group['brightness'].item(0)/20, #scale the size of the bubble; our bubble size is based on the brightness 
                color=colors[i], #the color of the bubbles in this group 
                line = dict(width=0.5, color='rgb(40,40,40)'), 
                sizemode='diameter'
                ), #sets the properties of the bubbles 
        name='{0}-{1}'.format(limits[i][0], limits[i][1])) #legend labels
    fires.append(fire)

NameError: name 'limits' is not defined

Let's see what our "fires" dictionary looks like: 

In [43]:
fires

[{'lat': array([34.466, 34.47 , 34.549, 34.415, 34.477]),
  'location': ['California'],
  'locationmode': 'USA-states',
  'lon': array([-119.078, -119.652, -119.35 , -119.41 , -119.079]),
  'marker': {'color': 'rgb(255,0,0)',
   'line': {'color': 'rgb(40,40,40)', 'width': 0.5},
   'size': 25.2,
   'sizemode': 'diameter'},
  'name': '0-5',
  'type': 'scattergeo'},
 {'lat': array([37.562, 37.56 , 37.557, 37.552, 37.55 ]),
  'location': ['California'],
  'locationmode': 'USA-states',
  'lon': array([-119.142, -119.13 , -119.116, -119.144, -119.13 ]),
  'marker': {'color': 'rgb(255,128,0)',
   'line': {'color': 'rgb(40,40,40)', 'width': 0.5},
   'size': 16.65,
   'sizemode': 'diameter'},
  'name': '5-10',
  'type': 'scattergeo'},
 {'lat': array([36.255, 34.339, 37.681, 37.212, 36.693]),
  'location': ['California'],
  'locationmode': 'USA-states',
  'lon': array([-118.412, -118.32 , -119.592, -119.418, -119.7  ]),
  'marker': {'color': 'rgb(255,255,0)',
   'line': {'color': 'rgb(40,40,40)'

Next, we will dictate what our figure layout should be. 

In [44]:
layout=dict(
        title='Range of Wildfire Brightnesses in Southern California from 09/01/2017 to 09/01/2018',
        showlegend=True,
        geo = dict(
            scope='CA', #doesn't do anything??
            projection=dict( type='albers usa'), #provides the gray USA map 
            center=dict(lon=-116.4194, lat=34.9592), #centers the map on the middle of SoCal when you first create the map
            zoom=6, #Zoom factor of map when you create it 
            showland = True,
            landcolor = 'rgb(217, 217, 217)',
            subunitwidth=1,
            countrywidth=1,
            subunitcolor="rgb(255, 255, 255)",
            countrycolor="rgb(255, 255, 255)"
        ),
    )
fig = dict(data=fires, layout=layout) #our figure with fires data and the layout we want 
py.iplot(fig, validate=False, filename='SoCal-FIRMS-bubble-map') #plot the data! 

In [16]:
dimmest['latitude']

array([36.082, 35.547, 36.297, 42.326, 39.603])

In [19]:
dimmest.take(0)['latitude'].item(0)

36.082

-  figure out how to just get california
-  Figure out label data w/ the date
-  legend title?

